In [ ]:
#using transformers_env

In [1]:
pip install peft

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.version.cuda)  # Should print 11.8

True
12.4


In [13]:
pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB 932.9 kB/s eta 0:46:47
     ---------------------------------------- 0.0/2.6 GB 882.6 kB/s eta 0:49:27
     ---------------------------------------- 0.0/2.6 GB 915.5 kB/s eta 0:47:40
     ---------------------------------------- 0.0/2.6 GB 972.7 kB/s eta 0:44:52
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:03
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:45
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:45
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:45
     --------------------

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    torch==2.0.1+cu118 from https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-win_amd64.whl#sha256=f58d75619bc96e4322343c030b893613701caa2d6db8017155da226c14171335:
        Expected sha256 f58d75619bc96e4322343c030b893613701caa2d6db8017155da226c14171335
             Got        d07c3a6a4050c083ea93a4430ba74a61e97cb20b1c117703828ae275c5b56f74



In [20]:
import torch
torch.cuda.empty_cache()

In [25]:
import torch
print(torch.cuda.memory_allocated() / 1024**2, "MB allocated")
print(torch.cuda.memory_reserved() / 1024**2, "MB reserved")

10310.3193359375 MB allocated
10436.0 MB reserved


model training code

In [19]:
import pandas as pd
import os
import torch
import shutil
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, TrainerCallback, EarlyStoppingCallback

# Check disk space
total, used, free = shutil.disk_usage('D:/')
if free < 5 * 1024**3:
    raise RuntimeError(f"Insufficient disk space: {free/1024**3:.2f}GB free, need ~5GB")

# Check write access
output_dir = r'D:/Python/dating coach/dating_coach/dating_coach_gpt2'
if not os.access(output_dir, os.W_OK):
    raise RuntimeError("No write access to output directory")

# Set PyTorch memory optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Memory logging callback
class MemoryLoggingCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        print(f"Step {state.global_step}: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

# Save checkpoint callback
class SaveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        try:
            model.save_pretrained(os.path.join(args.output_dir, f'checkpoint-{state.global_step}'))
            tokenizer.save_pretrained(os.path.join(args.output_dir, f'checkpoint-{state.global_step}'))
            print(f"Saved checkpoint at step {state.global_step}")
        except Exception as e:
            print(f"Checkpoint save failed at step {state.global_step}: {e}")

# Clean text
def clean_text(text):
    text = str(text).strip()
    if not text[-1] in [' ', '.', ',', '!', '?']:
        words = text.split()
        text = ' '.join(words[:-1]) if words else text
    return text.replace("your doing", "you're doing").replace("mistake guys make", "mistakes guys make")

# Read CSV data
data = []
csv_path = r'D:/Python/dating coach/formatted_data.csv'  # Adjust
if os.path.isfile(csv_path):
    df = pd.read_csv(csv_path)
    data = [{'title': row['title'], 'text': clean_text(row['text'])} for _, row in df.iterrows()]
else:
    for file in os.listdir(csv_path):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(csv_path, file))
            data.extend([{'title': row['title'], 'text': clean_text(row['text'])} for _, row in df.iterrows()])

# Format samples
formatted_samples = []
for item in data:
    sample = (
        f"[Scenario]: {item['title']}\n"
        f"{item['text']}\n"
        f"[Assistant]: For {item['title'].lower()}, keep it short and casual, like: 'Hey, great meeting you! Free this weekend?' Avoid long, eager messages as they can seem pushy."
    )
    formatted_samples.append(sample)

# # Subsample
# if len(formatted_samples) > 500:
#     formatted_samples = formatted_samples[:500]
dataset = Dataset.from_dict({'text': formatted_samples})

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
special_tokens = {'additional_special_tokens': ['[Scenario]', '[Assistant]']}
tokenizer.add_special_tokens(special_tokens)

# Load model
model = AutoModelForCausalLM.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))  # Fix lm_head.weight

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

# Clear GPU memory
torch.cuda.empty_cache()

# Tokenize dataset with labels
def tokenize(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128  # Reduced for speed
    )
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized
tokenized_dataset = dataset.map(tokenize, batched=True)

# Split dataset
train_dataset = tokenized_dataset.shuffle().select(range(int(0.8 * len(tokenized_dataset))))
eval_dataset = tokenized_dataset.shuffle().select(range(int(0.8 * len(tokenized_dataset)), len(tokenized_dataset)))

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=30,  # Up to 30 epochs
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,  # Lower for stability
    warmup_steps=100,  # Increased for large dataset
    max_grad_norm=1.0,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,  # Less frequent for large dataset
    eval_strategy='steps',
    eval_steps=500,  # Evaluate every 500 steps
    save_strategy='steps',
    save_steps=500,  # Save every 500 steps
    save_total_limit=3,
    save_safetensors=False,
    load_best_model_at_end=True,
    fp16=True,
    gradient_checkpointing=True,
    lr_scheduler_type='linear',
    report_to='none'
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[
        MemoryLoggingCallback(),
        SaveCheckpointCallback(),
        EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)
    ]
)

# Train
try:
    trainer.train()
except Exception as e:
    print(f"Training failed: {e}")
    try:
        close_file_handles(os.path.join(output_dir, 'final_new'))
        model.save_pretrained(os.path.join(output_dir, 'final_new'), save_safetensors=False)
        tokenizer.save_pretrained(os.path.join(output_dir, 'final_new'))
    except Exception as save_e:
        print(f"Final save failed: {save_e}")
    raise

# Save model and tokenizer
for attempt in range(3):
    try:
        close_file_handles(os.path.join(output_dir, 'final_new'))
        model.save_pretrained(os.path.join(output_dir, 'final_new'), save_safetensors=False)
        tokenizer.save_pretrained(os.path.join(output_dir, 'final_new'))
        print("Final model saved to final_new")
        break
    except Exception as e:
        print(f"Final save attempt {attempt+1} failed: {e}")
        time.sleep(5)
        if attempt == 2:
            print("Final save failed after 3 attempts")

Map: 100%|██████████| 1281/1281 [00:01<00:00, 867.71 examples/s]


Step 1: 2430.88 MB


Step,Training Loss,Validation Loss
500,2.579300,2.525645
1000,2.392700,2.389159
1500,2.299700,2.302292
2000,2.235100,2.244721
2500,2.189200,2.201106
3000,2.104400,2.172949


Step 2: 3385.56 MB
Step 3: 3385.56 MB
Step 4: 3385.56 MB
Step 5: 3385.56 MB
Step 6: 3385.56 MB
Step 7: 3385.56 MB
Step 8: 3385.56 MB
Step 9: 3385.56 MB
Step 10: 3385.56 MB
Step 11: 3385.56 MB
Step 12: 3385.56 MB
Step 13: 3385.56 MB
Step 14: 3385.56 MB
Step 15: 3385.56 MB
Step 16: 3385.56 MB
Step 17: 3385.56 MB
Step 18: 3385.56 MB
Step 19: 3385.56 MB
Step 20: 3385.56 MB
Step 21: 3385.56 MB
Step 22: 3385.56 MB
Step 23: 3385.56 MB
Step 24: 3385.56 MB
Step 25: 3385.56 MB
Step 26: 3385.56 MB
Step 27: 3385.56 MB
Step 28: 3385.56 MB
Step 29: 3385.56 MB
Step 30: 3385.56 MB
Step 31: 3385.56 MB
Step 32: 3385.56 MB
Step 33: 3385.56 MB
Step 34: 3385.56 MB
Step 35: 3385.56 MB
Step 36: 3385.56 MB
Step 37: 3385.56 MB
Step 38: 3385.56 MB
Step 39: 3385.56 MB
Step 40: 3385.56 MB
Step 41: 3385.56 MB
Step 42: 3385.56 MB
Step 43: 3385.56 MB
Step 44: 3385.56 MB
Step 45: 3385.56 MB
Step 46: 3385.56 MB
Step 47: 3385.56 MB
Step 48: 3385.56 MB
Step 49: 3385.56 MB
Step 50: 3385.56 MB
Step 51: 3385.56 MB
Step 52:

KeyboardInterrupt: 

interupted training continue code

In [ ]:
import pandas as pd
import os
import torch
import shutil
import psutil
import time
import numpy
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, TrainerCallback, EarlyStoppingCallback
# Override torch.load for trusted checkpoints
original_torch_load = torch.load
def custom_torch_load(*args, **kwargs):
    kwargs['weights_only'] = False  # Disable weights_only for trusted checkpoint
    return original_torch_load(*args, **kwargs)
torch.load = custom_torch_load
# Close open file handles
def close_file_handles(file_path):
    try:
        for proc in psutil.process_iter(['pid', 'name', 'open_files']):
            try:
                open_files = proc.info.get('open_files')
                if open_files is None:
                    continue
                for file in open_files:
                    if file.path.startswith(file_path):
                        print(f"Closing handle: {file.path} (PID: {proc.pid}, Name: {proc.info['name']})")
                        proc.terminate()
                        proc.wait(timeout=5)
            except (psutil.AccessDenied, psutil.NoSuchProcess, Exception) as e:
                print(f"Skipping process {proc.pid}: {e}")
    except Exception as e:
        print(f"Error closing handles: {e}")

# Check disk space
total, used, free = shutil.disk_usage('D:/')
if free < 10 * 1024**3:
    raise RuntimeError(f"Insufficient disk space: {free/1024**3:.2f}GB free, need ~10GB")

# Check write access
output_dir = r'D:/Python/dating coach/dating_coach/dating_coach_gpt2'
if not os.access(output_dir, os.W_OK):
    raise RuntimeError("No write access to output directory")

# Find latest checkpoint
checkpoints = [d for d in os.listdir(output_dir) if d.startswith('checkpoint-')]
if not checkpoints:
    raise RuntimeError("No checkpoints found. Must restart training.")
latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
checkpoint_path = os.path.join(output_dir, latest_checkpoint)
print(f"Resuming from: {checkpoint_path}")

# Set PyTorch memory optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Memory logging callback
class MemoryLoggingCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        print(f"Step {state.global_step}: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

# Save checkpoint callback
class SaveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        checkpoint_dir = os.path.join(args.output_dir, f'checkpoint-{state.global_step}')
        for attempt in range(3):
            try:
                close_file_handles(checkpoint_dir)
                model.save_pretrained(checkpoint_dir, save_safetensors=False)
                tokenizer.save_pretrained(checkpoint_dir)
                print(f"Saved checkpoint at step {state.global_step}")
                break
            except Exception as e:
                print(f"Checkpoint save attempt {attempt+1} failed: {e}")
                time.sleep(5)
                if attempt == 2:
                    print(f"Checkpoint save failed at step {state.global_step}")

# Clean text
def clean_text(text):
    text = str(text).strip()
    if not text[-1] in [' ', '.', ',', '!', '?']:
        words = text.split()
        text = ' '.join(words[:-1]) if words else text
    return text.replace("your doing", "you're doing").replace("mistake guys make", "mistakes guys make")

# Read CSV data
data = []
csv_path = r'D:/Python/dating coach/formatted_data.csv'  # Adjust
if os.path.isfile(csv_path):
    df = pd.read_csv(csv_path)
    data = [{'title': row['title'], 'text': clean_text(row['text'])} for _, row in df.iterrows()]
else:
    for file in os.listdir(csv_path):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(csv_path, file))
            data.extend([{'title': row['title'], 'text': clean_text(row['text'])} for _, row in df.iterrows()])

# Format samples with varied responses
responses = [
    "Try a concise approach: 'Hey, loved our chat! Free for coffee this weekend?' Avoid over-texting.",
    "Wait 1–2 days, then send a light message like: 'Hey, how’s it going?' Don’t sound too eager.",
    "Keep it playful: 'Hey, still thinking about that great convo! Up for a drink?' Avoid long texts."
]
import random
formatted_samples = []
for item in data:
    title = item['title'].lower()
    sample = (
        f"[Scenario]: {item['title']}\n"
        f"{item['text']}\n"
        f"[Assistant]: For {title}, {random.choice(responses)}"
    )
    formatted_samples.append(sample)

# Use full dataset
dataset = Dataset.from_dict({'text': formatted_samples})

# Load tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    tokenizer.pad_token = tokenizer.eos_token
except Exception as e:
    print(f"Tokenizer load failed: {e}")
    raise

# Load model from checkpoint
try:
    model = AutoModelForCausalLM.from_pretrained(checkpoint_path)
    model.gradient_checkpointing_enable()
except Exception as e:
    print(f"Model load failed: {e}")
    raise

torch.cuda.empty_cache()

# Tokenize dataset
def tokenize(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized
tokenized_dataset = dataset.map(tokenize, batched=True)

# Split dataset
train_dataset = tokenized_dataset.shuffle().select(range(int(0.8 * len(tokenized_dataset))))
eval_dataset = tokenized_dataset.shuffle().select(range(int(0.8 * len(tokenized_dataset)), len(tokenized_dataset)))

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=30,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=100,
    max_grad_norm=1.0,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=500,
    save_strategy='steps',
    save_steps=500,
    save_total_limit=3,
    save_safetensors=False,
    load_best_model_at_end=True,
    fp16=True,
    gradient_checkpointing=True,
    lr_scheduler_type='linear',
    report_to='none'
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[
        MemoryLoggingCallback(),
        SaveCheckpointCallback(),
        EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)
    ]
)

# Resume training
try:
    trainer.train(resume_from_checkpoint=checkpoint_path)
except Exception as e:
    print(f"Training failed: {e}")
    try:
        close_file_handles(os.path.join(output_dir, 'final_new'))
        model.save_pretrained(os.path.join(output_dir, 'final_new'), save_safetensors=False)
        tokenizer.save_pretrained(os.path.join(output_dir, 'final_new'))
    except Exception as save_e:
        print(f"Final save failed: {save_e}")
    raise

# Save final model
for attempt in range(3):
    try:
        close_file_handles(os.path.join(output_dir, 'final_new'))
        model.save_pretrained(os.path.join(output_dir, 'final_new'), save_safetensors=False)
        tokenizer.save_pretrained(os.path.join(output_dir, 'final_new'))
        print("Final model saved to final_new")
        break
    except Exception as e:
        print(f"Final save attempt {attempt+1} failed: {e}")
        time.sleep(5)
        if attempt == 2:
            print("Final save failed after 3 attempts")

Resuming from: D:/Python/dating coach/dating_coach/dating_coach_gpt2\checkpoint-3000


Map: 100%|██████████| 1281/1281 [00:00<00:00, 1376.49 examples/s]
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step 3001: 2894.09 MB


Step,Training Loss,Validation Loss


Step 3002: 2894.09 MB
Step 3003: 2894.09 MB
Step 3004: 2894.09 MB
Step 3005: 2894.09 MB
Step 3006: 2894.09 MB
Step 3007: 2894.09 MB
Step 3008: 2894.09 MB
Step 3009: 2894.09 MB
Step 3010: 2894.09 MB
Step 3011: 2894.09 MB


Inference code

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set device
device = torch.device('cuda:0')
# Clear GPU memory
torch.cuda.empty_cache()

# Load tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(r'D:/Python/dating coach/dating_coach/dating_coach_gpt2/final')
except Exception as e:
    print(f"Tokenizer load failed: {e}")
    exit(1)

# Load model
try:
    model = AutoModelForCausalLM.from_pretrained(r'D:/Python/dating coach/dating_coach/dating_coach_gpt2/final')
    model.to(device)  # Move model to GPU
    model.eval()  # Set to evaluation mode
except Exception as e:
    print(f"Model load failed: {e}")
    exit(1)

# Query
query = "User: What advice do you have for mistakes guys make after getting her number? [Scenario]: Mistakes Guys Make After Getting Her Number"

# Tokenize and move inputs to GPU
try:
    inputs = tokenizer(query, return_tensors='pt', truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move all tensors to cuda:0

    # Verify device
    for k, v in inputs.items():
        print(f"{k} device: {v.device}")
    print(f"Model device: {next(model.parameters()).device}")

    # Generate
    with torch.no_grad():  # Disable gradient computation
        outputs = model.generate(
            **inputs,
            max_length=150,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
            top_k=50, 
            repetition_penalty=1.2
        )

    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Response: {response}")
except Exception as e:
    print(f"Inference failed: {e}")

input_ids device: cuda:0
attention_mask device: cuda:0
Model device: cuda:0
Response: User: What advice do you have for mistakes guys make after getting her number? : Mistakes Guys Make After Getting Her Number 
User: What advice do you have for mistakes guys make after getting her number? 上: For mistakes guys make after getting her number, no one has ever been able to do it and so there's a lot of work to do. Try something like: No one has ever been able to do it and so there's a lot of work to do. Avoid things like: For mistakes guys make after getting her number, no one has ever been able to do it and so there's a lot of work to do. Avoid things like: 1. For girls who are not attracted to men, there
